In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

# path to json file that stores MFCCs and genre labels for each processed segment
DATA_PATH = "mfcc_data.json"

def load_data(data_path):
    """Loads training dataset from json file.

        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    # convert lists to numpy arrays
    X = np.array(data["mfccs"])
    y = np.array(data["labels"])
    genres = data["genres"]

    print("Data succesfully loaded!")

    return  X, y, genres

In [2]:
# load data
X, y, genres = load_data(DATA_PATH)

Data succesfully loaded!


In [3]:
# create train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [4]:
# build network topology
model = keras.Sequential([

    # input layer
    keras.layers.Flatten(input_shape=(X.shape[1], X.shape[2])),

    # 1st dense layer
    keras.layers.Dense(512, activation='relu'),

    # 2nd dense layer
    keras.layers.Dense(256, activation='relu'),

    # 3rd dense layer
    keras.layers.Dense(64, activation='relu'),

    # output layer
    keras.layers.Dense(10, activation='softmax')
])

# compile model
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 8620)              0         
                                                                 
 dense (Dense)               (None, 512)               4413952   
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 64)                16448     
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 4,562,378
Trainable params: 4,562,378
Non-trainable params: 0
_________________________________________________________________


In [5]:
# train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=50)

Epoch 1/50
22/22 [==============================] - 2s 53ms/step - loss: 16.7259 - accuracy: 0.2504 - val_loss: 7.3154 - val_accuracy: 0.2467
Epoch 2/50
22/22 [==============================] - 1s 46ms/step - loss: 5.8131 - accuracy: 0.3848 - val_loss: 6.7548 - val_accuracy: 0.2967
Epoch 3/50
22/22 [==============================] - 1s 48ms/step - loss: 3.1805 - accuracy: 0.5150 - val_loss: 5.9016 - val_accuracy: 0.3433
Epoch 4/50
22/22 [==============================] - 1s 48ms/step - loss: 1.7316 - accuracy: 0.6524 - val_loss: 4.7343 - val_accuracy: 0.3367
Epoch 5/50
22/22 [==============================] - 1s 48ms/step - loss: 1.0196 - accuracy: 0.7167 - val_loss: 4.5832 - val_accuracy: 0.3400
Epoch 6/50
22/22 [==============================] - 1s 49ms/step - loss: 0.5947 - accuracy: 0.8355 - val_loss: 4.8719 - val_accuracy: 0.3300
Epoch 7/50
22/22 [==============================] - 1s 48ms/step - loss: 0.4584 - accuracy: 0.8484 - val_loss: 4.2524 - val_accuracy: 0.4200
Epoch 8/50
2

In [6]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=64)
print("test loss, test acc:", results)

Evaluate on test data
5/5 [==============================] - 0s 5ms/step - loss: 5.2069 - accuracy: 0.4233
test loss, test acc: [5.206914901733398, 0.4233333468437195]


In [7]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for tests")
predictions = model.predict(X_test)
correct = 0
for i in range(predictions.shape[0]):
    max_confidence = max(predictions[i])
    pred = np.argmax(predictions[i])
    actual = y_test[i]
    if pred == actual:
        correct += 1
    print('Prediction:', genres[pred], '\tActual:', genres[actual], '\nConfidence:', max(predictions[i]),'\n')

Generate predictions for tests
10/10 [==============================] - 0s 5ms/step
Prediction: classical 	Actual: classical 
Confidence: 0.99663806 

Prediction: blues 	Actual: blues 
Confidence: 0.99468476 

Prediction: hiphop 	Actual: rock 
Confidence: 0.954318 

Prediction: pop 	Actual: pop 
Confidence: 0.85295016 

Prediction: disco 	Actual: rock 
Confidence: 0.9998958 

Prediction: disco 	Actual: jazz 
Confidence: 0.633555 

Prediction: rock 	Actual: blues 
Confidence: 0.9962411 

Prediction: jazz 	Actual: hiphop 
Confidence: 0.9042243 

Prediction: jazz 	Actual: country 
Confidence: 0.9466938 

Prediction: metal 	Actual: metal 
Confidence: 0.9998325 

Prediction: reggae 	Actual: reggae 
Confidence: 0.79727894 

Prediction: reggae 	Actual: reggae 
Confidence: 0.89664125 

Prediction: country 	Actual: jazz 
Confidence: 0.9999888 

Prediction: reggae 	Actual: jazz 
Confidence: 0.8187228 

Prediction: jazz 	Actual: country 
Confidence: 0.8835665 

Prediction: hiphop 	Actual: country

In [8]:
print(correct / predictions.shape[0] * 100,'% correctly predicted')

42.333333333333336 % correctly predicted
